<a href="https://colab.research.google.com/github/d3rp3tt3/AI-Trivia-ChatBot-Project-3/blob/jenn-embeddings/jenn_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Jenn Allen's working file for the AI Chatbook - work in progress

In [ ]:
# Install dependencies
!pip install transformers
!pip install accelerate
!pip install sentence-transformers
!pip install chromadb
!pip install langchain
!pip install langchain-community
!pip install pypdf
# Test

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.1 MB/s eta 0:00:00


In [ ]:
# Import modules
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

In [ ]:
# Load Phi 3 model
model_kwargs = {'device': 'cuda'}
embeddings = HuggingFaceEmbeddings(model_kwargs=model_kwargs)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct", torch_dtype='auto', trust_remote_code=True)

pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, device=-1, max_new_tokens=300)
llm = HuggingFacePipeline(pipeline=pipe)

<ipython-input-5-beebfb740d9b>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_kwargs=model_kwargs)
<ipython-input-5-beebfb740d9b>:3: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings(model_kwargs=model_kwargs)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, cre

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

<ipython-input-5-beebfb740d9b>:9: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [ ]:
# Load data

import csv

def load_trivia_questions(filename):
    trivia_questions = {}
    with open(filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            category = row['Category']
            question = row['Question']
            answer = row['Answer']
            if category not in trivia_questions:
                trivia_questions[category] = []
            trivia_questions[category].append({"question": question, "answer": answer})
    return trivia_questions

trivia_questions = load_trivia_questions('test_question_set.csv')

In [ ]:
# Create embeddings for the trivia questions, for use in RAG

def create_embeddings(questions):
    embeddings = []
    for category, question_list in questions.items():
        for question in question_list:
            question_text = question["question"]
            embedding = embeddings.embed(question_text)
            embeddings.append((question_text, embedding))
    return embeddings

# embeddings = create_embeddings(trivia_questions)

In [ ]:
print(type(test[0]))


NameError: name 'test' is not defined

In [ ]:
# Set up RAG
from langchain.vectorstores import Chroma

db = Chroma.from_documents(embeddings, embedding=embeddings, persist_directory="trivia_index")
db.persist()

retriever = db.as_retriever(search_kwargs={"k": 1})  # Retrieve only the most relevant document

AttributeError: 'tuple' object has no attribute 'page_content'

In [ ]:
# Create custom prompt template for the chatbot

qna_prompt_template = """
<system>
You are a trivia bot. When asked a question, choose a category from the list provided and select a trivia question from your knowledge base in that category. Provide the answer to the trivia question.

Available categories: {categories}

User query: Ask me a trivia question from the category you choose.

Your trivia question database:
{question_database}
Y4:0"""